# <span style='color:#ff5f27'> Initialization </span>

### Hopsworks Settings

In [1]:
import sys
from pathlib import Path
import os

def is_google_colab() -> bool:
    if "google.colab" in str(get_ipython()):
        return True
    return False

def clone_repository() -> None:
    !git clone https://github.com/featurestorebook/mlfs-book.git
    %cd mlfs-book

def install_dependencies() -> None:
    !pip install --upgrade uv
    !uv pip install --all-extras --system --requirement pyproject.toml


if is_google_colab():
    clone_repository()
    install_dependencies()
    root_dir = str(Path().absolute())
    print("Google Colab environment")
else:
    root_dir = Path().absolute()
    # Strip ~/notebooks/ccfraud from PYTHON_PATH if notebook started in one of these subdirectories
    if root_dir.parts[-1:] == ('airquality',):
        root_dir = Path(*root_dir.parts[:-1])
    if root_dir.parts[-1:] == ('notebooks',):
        root_dir = Path(*root_dir.parts[:-1])
    root_dir = str(root_dir) 
    print("Local environment")

# Add the root directory to the `PYTHONPATH` to use the `recsys` Python module from the notebook.
if root_dir not in sys.path:
    sys.path.append(root_dir)
print(f"Added the following directory to the PYTHONPATH: {root_dir}")
    
# Read the API keys and configuration variables from the file <root_dir>/.env
from mlfs import config
if os.path.exists(f"{root_dir}/.env"):
    settings = config.HopsworksSettings(_env_file=f"{root_dir}/.env")

Local environment
Added the following directory to the PYTHONPATH: /home/federica_lorenzini/sml-bike-sharing
HopsworksSettings initialized!


### Imports

In [2]:
import datetime
import pandas as pd
from xgboost import XGBRegressor
import hopsworks
import json
from mlfs import util

2026-01-11 15:58:45,997 WARNING: DeprecationWarning: 'parseString' deprecated - use 'parse_string'

2026-01-11 15:58:46,000 WARNING: DeprecationWarning: 'resetCache' deprecated - use 'reset_cache'

2026-01-11 15:58:46,175 WARNING: DeprecationWarning: 'enablePackrat' deprecated - use 'enable_packrat'

2026-01-11 15:58:46,246 WARNING: In /home/federica_lorenzini/sml-bike-sharing/.venv/lib/python3.10/site-packages/matplotlib/mpl-data/stylelib/classic.mplstyle: 'parseString' deprecated - use 'parse_string'
2026-01-11 15:58:46,249 WARNING: In /home/federica_lorenzini/sml-bike-sharing/.venv/lib/python3.10/site-packages/matplotlib/mpl-data/stylelib/classic.mplstyle: 'resetCache' deprecated - use 'reset_cache'


# <span style='color:#ff5f27'> Retrieve Data </span>

### Retrieve Metadata

In [3]:
today = datetime.datetime.now() - datetime.timedelta(0)
tomorrow = today + datetime.timedelta(days = 1)
today

datetime.datetime(2026, 1, 11, 15, 58, 46, 857997)

In [4]:
# Check if HOPSWORKS_API_KEY env variable is set or if it is set in ~/.env
if settings.HOPSWORKS_API_KEY is not None:
    api_key = settings.HOPSWORKS_API_KEY.get_secret_value()
    os.environ['HOPSWORKS_API_KEY'] = api_key
project = hopsworks.login()
fs = project.get_feature_store() 

secrets = hopsworks.get_secrets_api()
location_str = secrets.get_secret("BIKES_LOCATION_JSON").value
location = json.loads(location_str)

country=location['country']
city=location['city']
latitude=location['latitude']
longitude=location['longitude']
today = datetime.date.today()
yesterday = today - datetime.timedelta(days=1)

print(f"City: {city}")
print(f"Contry: {country}")
print(f"Latitude: {latitude}")
print(f"Longitude: {longitude}")
print(f"Day: {today}")

2026-01-11 15:58:46,890 INFO: Initializing external client
2026-01-11 15:58:46,893 INFO: Base URL: https://c.app.hopsworks.ai:443
2026-01-11 15:58:47,874 WARNING: UserWarning: The installed hopsworks client version 4.4.2 may not be compatible with the connected Hopsworks backend version 4.2.2. 
To ensure compatibility please install the latest bug fix release matching the minor version of your backend (4.2) by running 'pip install hopsworks==4.2.*'



2026-01-11 15:58:49,002 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1279175
City: Trento
Contry: Italy
Latitude: 46.07
Longitude: 11.12
Day: 2026-01-11


# <span style='color:#ff5f27'> Retrieve the Model </span>

### Download the Model From Model Registry

In [5]:
mr = project.get_model_registry()

retrieved_model = mr.get_model(
    name="trento_bikes_xgboost_model",
    version=3,
)

fv = retrieved_model.get_feature_view()

# Download the saved model artifacts to a local directory
saved_model_dir = retrieved_model.download()

2026-01-11 15:58:55,745 INFO: Initializing for batch retrieval of feature vectors


Downloading: 0.000%|          | 0/1085921 elapsed<00:00 remaining<?

Downloading: 0.000%|          | 0/20189 elapsed<00:00 remaining<?

### Loading the XGBoost Regressor Model

In [6]:
retrieved_xgboost_model = XGBRegressor(enable_categorical=True)
retrieved_xgboost_model.load_model(saved_model_dir + "/trento_bikes_xgboost_model.json")

# Displaying the retrieved XGBoost regressor model
retrieved_xgboost_model

XGBRegressor(base_score='3.9057984E0', booster='gbtree', callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=True, eval_metric=None,
             feature_types=['c', 'c', 'float', 'float', 'float', 'float'],
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

# <span style='color:#ff5f27'> Retrieve Data </span>

### Get Weather Forecast

In [7]:
weather_fg = fs.get_feature_group(
    name='weather_trento',
    version=2,
)

# To avoid hourly discrepancies, we filter dates > than yesterday (rather than >= today)
weather_data = weather_fg.filter(weather_fg.date > today-datetime.timedelta(days=1)).read()

weather_data = weather_data.sort_values(by=['date'], ignore_index=True)
weather_data['days_before_forecast'] = range(1, len(weather_data)+1)
weather_data.head(10)

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.93s) 


,date,weather_code,apparent_temperature_mean,daylight_duration,precipitation_sum,wind_speed_10m_max,city,days_before_forecast
0,2026-01-11 00:00:00+00:00,2,-2.276061,32165.906250,0.000000,5.991594,Trento,1
1,2026-01-12 00:00:00+00:00,3,-3.405265,32267.652344,0.000000,3.319036,Trento,2
2,2026-01-13 00:00:00+00:00,3,0.622029,32373.125000,0.000000,2.276840,Trento,3
3,2026-01-14 00:00:00+00:00,51,3.051547,32482.150391,0.400000,2.902413,Trento,4
4,2026-01-15 00:00:00+00:00,51,4.228703,32594.542969,0.400000,2.545584,Trento,5
5,2026-01-16 00:00:00+00:00,51,3.586735,32710.150391,0.300000,3.600000,Trento,6
6,2026-01-17 00:00:00+00:00,53,4.472055,32828.796875,6.799999,3.396233,Trento,7


### Get Today's Bikes

In [8]:
bikes_fg = fs.get_feature_group(
    name='bikes_trento',
    version=2,
)

bikes_today = bikes_fg.filter(bikes_fg.date > (today-datetime.timedelta(days=1))).read()
bikes_today.head()

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (1.15s) 


,tag,id,name,latitude,longitude,bikes,free,extra,date,country,city
0,e-motion-trento,0b05bb893eb6aa4ebc6d49b0f07feb0b,10.18 Vannetti,46.072997,11.121911,0,0,"{'uid': '1494', 'number': 1018, 'status': 'off...",2026-01-10 23:26:39.646643+00:00,Italy,Trento
1,e-motion-trento,69223b267f0803ecfec3e334ac0f34ef,23.02 Piazzo,45.922140,11.033578,0,3,"{'uid': '1764', 'number': 2302, 'status': 'off...",2026-01-10 23:26:39.647628+00:00,Italy,Trento
2,e-motion-trento,1132448419fefa8717d49aa4eb8f5ac1,10.25 Studentato Mayer,46.072743,11.117380,3,9,"{'uid': '1718', 'number': 1025, 'status': 'onl...",2026-01-10 23:26:39.646738+00:00,Italy,Trento
3,e-motion-trento,7672409ffc0a15bf70d9ebd0feef635e,20.21 Polo Meccatronica,45.891218,11.032655,2,9,"{'uid': '1827', 'number': 2021, 'status': 'off...",2026-01-10 23:26:39.648161+00:00,Italy,Trento
4,e-motion-trento,42e643f54b882e0aae5b063a3593c85b,10.24 Campo CONI,46.079633,11.117579,1,11,"{'uid': '1717', 'number': 1024, 'status': 'onl...",2026-01-10 23:26:39.646684+00:00,Italy,Trento


In [9]:
bikes_today.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 170 entries, 0 to 169
Data columns (total 11 columns):
 #   Column     Non-Null Count  Dtype                  
---  ------     --------------  -----                  
 0   tag        170 non-null    object                 
 1   id         170 non-null    object                 
 2   name       170 non-null    object                 
 3   latitude   170 non-null    float64                
 4   longitude  170 non-null    float64                
 5   bikes      170 non-null    int32                  
 6   free       170 non-null    int32                  
 7   extra      170 non-null    object                 
 8   date       170 non-null    datetime64[us, Etc/UTC]
 9   country    170 non-null    object                 
 10  city       170 non-null    object                 
dtypes: datetime64[us, Etc/UTC](1), float64(2), int32(2), object(6)
memory usage: 13.4+ KB


# <span style='color:#ff5f27'> Making the Predictions </span>

### Prepare the Data to Fed to the Model

In [10]:
# Just in case there are more rows retrieved
bikes_today = bikes_today.sort_values(by=['date'], ascending=False)

joint_df = pd.merge(
    bikes_today[['name', 'city']],
    weather_data[['date', 'weather_code', 'apparent_temperature_mean', 'daylight_duration', 'precipitation_sum', 'wind_speed_10m_max', 'city', 'days_before_forecast']],
    on='city', how='left'
)
joint_df = joint_df.drop(columns=['city'])
joint_df['name'] = joint_df['name'].astype("category")
joint_df['weather_code'] = joint_df['weather_code'].astype("category")

joint_df.head(10)


,name,date,weather_code,apparent_temperature_mean,daylight_duration,precipitation_sum,wind_speed_10m_max,days_before_forecast
0,10.42 Ponte dei Cavalleggeri,2026-01-11 00:00:00+00:00,2,-2.276061,32165.906250,0.000000,5.991594,1
1,10.42 Ponte dei Cavalleggeri,2026-01-12 00:00:00+00:00,3,-3.405265,32267.652344,0.000000,3.319036,2
2,10.42 Ponte dei Cavalleggeri,2026-01-13 00:00:00+00:00,3,0.622029,32373.125000,0.000000,2.276840,3
3,10.42 Ponte dei Cavalleggeri,2026-01-14 00:00:00+00:00,51,3.051547,32482.150391,0.400000,2.902413,4
4,10.42 Ponte dei Cavalleggeri,2026-01-15 00:00:00+00:00,51,4.228703,32594.542969,0.400000,2.545584,5
5,10.42 Ponte dei Cavalleggeri,2026-01-16 00:00:00+00:00,51,3.586735,32710.150391,0.300000,3.600000,6
6,10.42 Ponte dei Cavalleggeri,2026-01-17 00:00:00+00:00,53,4.472055,32828.796875,6.799999,3.396233,7
7,20.24 Ex Marangoni,2026-01-11 00:00:00+00:00,2,-2.276061,32165.906250,0.000000,5.991594,1
8,20.24 Ex Marangoni,2026-01-12 00:00:00+00:00,3,-3.405265,32267.652344,0.000000,3.319036,2
9,20.24 Ex Marangoni,2026-01-13 00:00:00+00:00,3,0.622029,32373.125000,0.000000,2.276840,3


In [11]:
joint_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1190 entries, 0 to 1189
Data columns (total 8 columns):
 #   Column                     Non-Null Count  Dtype                  
---  ------                     --------------  -----                  
 0   name                       1190 non-null   category               
 1   date                       1190 non-null   datetime64[us, Etc/UTC]
 2   weather_code               1190 non-null   category               
 3   apparent_temperature_mean  1190 non-null   float32                
 4   daylight_duration          1190 non-null   float32                
 5   precipitation_sum          1190 non-null   float32                
 6   wind_speed_10m_max         1190 non-null   float32                
 7   days_before_forecast       1190 non-null   int64                  
dtypes: category(2), datetime64[us, Etc/UTC](1), float32(4), int64(1)
memory usage: 42.5 KB


### Make the Predictions

In [12]:
joint_df['predicted_bikes'] = retrieved_xgboost_model.predict(
    joint_df[['name', 'weather_code', 'apparent_temperature_mean', 'daylight_duration', 'precipitation_sum', 'wind_speed_10m_max']]
)

joint_df['city'] = city
joint_df['country'] = country
joint_df = joint_df.sort_values(by=['date'])

joint_df.head()

,name,date,weather_code,apparent_temperature_mean,daylight_duration,precipitation_sum,wind_speed_10m_max,days_before_forecast,predicted_bikes,city,country
0,10.42 Ponte dei Cavalleggeri,2026-01-11 00:00:00+00:00,2,-2.276061,32165.90625,0.0,5.991594,1,2.539933,Trento,Italy
210,16.01 Aldeno,2026-01-11 00:00:00+00:00,2,-2.276061,32165.90625,0.0,5.991594,1,1.207573,Trento,Italy
749,20.05 Zona Industriale,2026-01-11 00:00:00+00:00,2,-2.276061,32165.90625,0.0,5.991594,1,1.543687,Trento,Italy
742,20.06 Paoli,2026-01-11 00:00:00+00:00,2,-2.276061,32165.90625,0.0,5.991594,1,3.473789,Trento,Italy
217,10.41 Piedicastello - Ex Italcementi,2026-01-11 00:00:00+00:00,2,-2.276061,32165.90625,0.0,5.991594,1,2.115619,Trento,Italy


In [13]:
joint_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1190 entries, 0 to 1189
Data columns (total 11 columns):
 #   Column                     Non-Null Count  Dtype                  
---  ------                     --------------  -----                  
 0   name                       1190 non-null   category               
 1   date                       1190 non-null   datetime64[us, Etc/UTC]
 2   weather_code               1190 non-null   category               
 3   apparent_temperature_mean  1190 non-null   float32                
 4   daylight_duration          1190 non-null   float32                
 5   precipitation_sum          1190 non-null   float32                
 6   wind_speed_10m_max         1190 non-null   float32                
 7   days_before_forecast       1190 non-null   int64                  
 8   predicted_bikes            1190 non-null   float32                
 9   city                       1190 non-null   object                 
 10  country                    11

# <span style='color:#ff5f27'> Create Forecast Graph </span>

### Obtain Metadata 

In [14]:
metadata = pd.DataFrame()
metadata['name'] = joint_df['name'].drop_duplicates()  
metadata.head()

,name
0,10.42 Ponte dei Cavalleggeri
210,16.01 Aldeno
749,20.05 Zona Industriale
742,20.06 Paoli
217,10.41 Piedicastello - Ex Italcementi


In [15]:
metadata.info()

<class 'pandas.core.frame.DataFrame'>
Index: 85 entries, 0 to 441
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype   
---  ------  --------------  -----   
 0   name    85 non-null     category
dtypes: category(1)
memory usage: 3.5 KB


### Plot the Forecasts

In [16]:
for i in range(metadata.shape[0]):
    name = metadata.iloc[i]['name']
    pred_file_path = f"{root_dir}/dashboard/img/bikes_forecast_"+name+".png"
    joint_df_i = joint_df[joint_df['name'] == name]
    plt = util.plot_bikes_forecast(city, name, joint_df_i, pred_file_path)
    plt.close()

2026-01-11 15:59:05,849 WARNING: DeprecationWarning: backend2gui is deprecated since IPython 8.24, backends are managed in matplotlib and can be externally registered.



# <span style='color:#ff5f27'> Upload Monitor Feature Group </span>

In [17]:
monitor_fg = fs.get_or_create_feature_group(
    name='bikes_predictions_trento',
    description='Bike Prediction Monitoring for Trento',
    version=2,
    primary_key=['city','name','date','days_before_forecast'],
    event_time="date"
)

Convert `weather_code` to integer as Hopsworks doesn't support categories

In [18]:
joint_df['weather_code'] = joint_df['weather_code'].astype('int32')

In [19]:
monitor_fg.insert(joint_df, wait=True)

Uploading Dataframe: 100.00% |██████████| Rows 1190/1190 | Elapsed Time: 00:02 | Remaining Time: 00:00


Launching job: bikes_predictions_trento_2_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1279175/jobs/named/bikes_predictions_trento_2_offline_fg_materialization/executions
2026-01-11 15:59:49,330 INFO: Waiting for execution to finish. Current state: SUBMITTED. Final status: UNDEFINED
2026-01-11 15:59:52,558 INFO: Waiting for execution to finish. Current state: RUNNING. Final status: UNDEFINED
2026-01-11 16:01:49,595 INFO: Waiting for execution to finish. Current state: AGGREGATING_LOGS. Final status: SUCCEEDED
2026-01-11 16:01:49,800 INFO: Waiting for log aggregation to finish.
2026-01-11 16:02:08,970 INFO: Execution finished successfully.


(Job('bikes_predictions_trento_2_offline_fg_materialization', 'SPARK'), None)

In [20]:
monitor_fg.update_feature_description("name", "Sensor Name")
monitor_fg.update_feature_description("date", "Measurement Day")
monitor_fg.update_feature_description("weather_code", "Weather Condition Numeric Code (WMO)")
monitor_fg.update_feature_description("apparent_temperature_mean", "Apparent Temperature, Mean of the Day")
monitor_fg.update_feature_description("daylight_duration", "Seconds of Sun for the Day")
monitor_fg.update_feature_description("precipitation_sum", "Total Precipitation for the Day")
monitor_fg.update_feature_description("wind_speed_10m_max", "Wind Speed")
monitor_fg.update_feature_description("days_before_forecast", "Days before the forecast")
monitor_fg.update_feature_description("predicted_bikes", "Number of available bikes by the model")
monitor_fg.update_feature_description("city", "City of Measurement")
monitor_fg.update_feature_description("country", "Country of Measurement")

# <span style='color:#ff5f27'> Create Hindcast </span>

### Retrieve Monitor Data

In [21]:
# We will create a hindcast chart for only the forecasts made 1 day beforehand
monitoring_df = monitor_fg.filter(monitor_fg.days_before_forecast == 1).read()
monitoring_df.head()

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (6.63s) 


,name,date,weather_code,apparent_temperature_mean,daylight_duration,precipitation_sum,wind_speed_10m_max,days_before_forecast,predicted_bikes,city,country
0,10.29 Gardolo - ex segheria,2026-01-11 00:00:00+00:00,2,-2.276061,32165.90625,0.0,5.991594,1,3.171497,Trento,Italy
1,20.10 Noriglio,2026-01-11 00:00:00+00:00,2,-2.276061,32165.90625,0.0,5.991594,1,1.710842,Trento,Italy
2,22.01 Pomarolo,2026-01-11 00:00:00+00:00,2,-2.276061,32165.90625,0.0,5.991594,1,1.888657,Trento,Italy
3,20.20 Cimitero,2026-01-11 00:00:00+00:00,2,-2.276061,32165.90625,0.0,5.991594,1,1.257085,Trento,Italy
4,20.13 Quercia,2026-01-11 00:00:00+00:00,2,-2.276061,32165.90625,0.0,5.991594,1,2.463043,Trento,Italy


### Retrieve Bikes Data

In [22]:
bikes_fg = fs.get_feature_group(name='bikes_trento', version=2)
bikes_df = bikes_fg.read()
bikes_df.head()

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (9.67s) 


,tag,id,name,latitude,longitude,bikes,free,extra,date,country,city
0,e-motion-trento,7742688aceb9aef4e0e1ff90a1fe1df1,20.24 Ex Marangoni,45.873807,11.029851,1,5,"{""uid"":""1837"",""number"":2024,""status"":""online"",...",2025-06-25 19:10:58+00:00,Italy,Trento
1,e-motion-trento,34cffd58626100057673cd2b5eb20253,10.19 Cimitero,46.063654,11.119203,2,8,"{""uid"":""1495"",""number"":1019,""status"":""online"",...",2025-10-21 11:08:39+00:00,Italy,Trento
2,e-motion-trento,98761742d3691fe85ea3a03d6f186f2c,20.18 Baldresca,45.880535,11.028777,4,2,"{""uid"":""1782"",""number"":2018,""status"":""online"",...",2025-05-28 15:36:18+00:00,Italy,Trento
3,e-motion-trento,bbb2c885186be3c3b372d4ebefcabe5d,20.12 Rosmini,45.890947,11.043008,1,5,"{""uid"":""1776"",""number"":2012,""status"":""online"",...",2025-09-01 08:36:21+00:00,Italy,Trento
4,e-motion-trento,c33ab7ecd3239912fa8ac789bba114ad,20.14 Orvea,45.871987,11.020448,4,8,"{""uid"":""1778"",""number"":2014,""status"":""online"",...",2025-08-04 03:23:53+00:00,Italy,Trento


### Merge the Two

In [23]:
outcome_df = bikes_df[['date', 'bikes', 'name']]
outcome_df['date'] = outcome_df['date'].apply(lambda x: pd.Timestamp(x).replace(hour=0, minute=0, second=0, microsecond=0))
preds_df =  monitoring_df[['date', 'predicted_bikes', 'name']]

hindcast_df = pd.merge(preds_df, outcome_df, on=['date','name'])
hindcast_df = hindcast_df.sort_values(by=['date'])
hindcast_df

# If there are no outcomes for predictions yet, generate some predictions/outcomes from existing data
if len(hindcast_df) == 0:
    print("Using Prediction Backfill")
    hindcast_df = util.backfill_predictions_for_monitoring_bikes(weather_fg, bikes_df, monitor_fg, retrieved_xgboost_model)
    hindcast_df = hindcast_df[['date', 'predicted_bikes', 'name', 'bikes']]

hindcast_df.head(10)

2026-01-11 16:02:48,260 WARNING: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,date,predicted_bikes,name,bikes
169,2026-01-10 00:00:00+00:00,4.569208,20.19 Follone,4
108,2026-01-10 00:00:00+00:00,3.462941,12.03 Stazione TN-Male`,4
109,2026-01-10 00:00:00+00:00,5.305099,10.08 Universita`,7
110,2026-01-10 00:00:00+00:00,0.878969,12.01 Cavalleggeri,2
111,2026-01-10 00:00:00+00:00,5.311347,10.12 Azienda Sanitaria,2
112,2026-01-10 00:00:00+00:00,5.978068,10.27 Clarina,5
113,2026-01-10 00:00:00+00:00,4.314265,10.24 Campo CONI,1
114,2026-01-10 00:00:00+00:00,2.258966,20.22 Lizzanella,0
115,2026-01-10 00:00:00+00:00,3.699264,10.15 Muse,2
116,2026-01-10 00:00:00+00:00,3.120400,11.03 Fiera,3


In [24]:
hindcast_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 170 entries, 169 to 84
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype                  
---  ------           --------------  -----                  
 0   date             170 non-null    datetime64[us, Etc/UTC]
 1   predicted_bikes  170 non-null    float32                
 2   name             170 non-null    object                 
 3   bikes            170 non-null    int32                  
dtypes: datetime64[us, Etc/UTC](1), float32(1), int32(1), object(1)
memory usage: 5.3+ KB


### Plot Hindcast

In [25]:
for i in range(metadata.shape[0]):
    name = metadata.iloc[i]['name']
    hindcast_file_path = f"{root_dir}/dashboard/img/bikes_hindcast_1day_"+name+".png"
    hindcast_df_i = hindcast_df[hindcast_df['name'] == name]
    plt = util.plot_bikes_forecast(city, name, hindcast_df_i, hindcast_file_path, hindcast=True)
    plt.close()

# <span style='color:#ff5f27'> Upload Pictures </span>

In [ ]:
dataset_api = project.get_dataset_api()
str_today = today.strftime("%Y-%m-%d")
folder_name = "bikes_v2"

if dataset_api.exists(f"Resources/{folder_name}") == False:
    dataset_api.mkdir(f"Resources/{folder_name}")

for i in range(metadata.shape[0]):
    name = metadata.iloc[i]['name']

    pred_file_path = f"{root_dir}/dashboard/img/bikes_forecast_"+name+".png"
    hindcast_file_path = f"{root_dir}/dashboard/img/bikes_hindcast_1day_"+name+".png"

    dataset_api.upload(pred_file_path, f"Resources/{folder_name}/{city}_{name}_{str_today}", overwrite=True)
    dataset_api.upload(hindcast_file_path, f"Resources/{folder_name}/{city}_{name}_{str_today}", overwrite=True)

proj_url = project.get_url()
print(f"See images in Hopsworks here: {proj_url}/settings/fb/path/Resources/{folder_name}")

Uploading /home/federica_lorenzini/sml-bike-sharing/dashboard/img/bikes_forecast_10.42 Ponte dei Cavalleggeri.…

Uploading /home/federica_lorenzini/sml-bike-sharing/dashboard/img/bikes_hindcast_1day_10.42 Ponte dei Cavalleg…

Uploading /home/federica_lorenzini/sml-bike-sharing/dashboard/img/bikes_forecast_16.01 Aldeno.png: 0.000%|    …

Uploading /home/federica_lorenzini/sml-bike-sharing/dashboard/img/bikes_hindcast_1day_16.01 Aldeno.png: 0.000%…

Uploading /home/federica_lorenzini/sml-bike-sharing/dashboard/img/bikes_forecast_20.05 Zona Industriale.png: 0…

...

Uploading /home/federica_lorenzini/sml-bike-sharing/dashboard/img/bikes_forecast_10.28 Barbacovi.png: 0.000%| …

Uploading /home/federica_lorenzini/sml-bike-sharing/dashboard/img/bikes_hindcast_1day_10.28 Barbacovi.png: 0.0…

Uploading /home/federica_lorenzini/sml-bike-sharing/dashboard/img/bikes_forecast_10.25 Studentato Mayer.png: 0…

Uploading /home/federica_lorenzini/sml-bike-sharing/dashboard/img/bikes_hindcast_1day_10.25 Studentato Mayer.p…

Uploading /home/federica_lorenzini/sml-bike-sharing/dashboard/img/bikes_forecast_10.27 Clarina.png: 0.000%|   …